In [1]:
from keras.utils import np_utils
import numpy as np
from matplotlib import pyplot
import seaborn as sns
%matplotlib inline

# Display progress logs on stdout
#import logging
#logging.basicConfig(level=logging.DEBUG,
#                    format='%(asctime)s %(levelname)s %(message)s')

from sklearn.datasets import load_files
import pandas as pd
pd.set_option("display.max_colwidth", 75)

import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config = config)

Using TensorFlow backend.
/home/tutysara/anaconda2/envs/dog-project/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
# define function to load train, test, and validation datasets
basedir="/media/hdd/datastore/t4sa"
train_idx_path = basedir+ "/b-t4sa_train.txt"
valid_idx_path = basedir+ "/b-t4sa_val.txt"
test_idx_path = basedir+ "/b-t4sa_test.txt"


col_names = ["X", "y"]
percent = 0.005
train_data_df = pd.read_csv(train_idx_path, sep=" ", header=None, names=col_names)
valid_data_df = pd.read_csv(valid_idx_path, sep=" ", header=None, names=col_names)
test_data_df = pd.read_csv(test_idx_path, sep=" ", header=None, names=col_names)

train_data_df = train_data_df[:int(train_data_df.shape[0]*percent)]
valid_data_df = valid_data_df[:int(valid_data_df.shape[0]*percent)]
test_data_df = test_data_df[:int(test_data_df.shape[0]*percent)]

In [3]:
test_data_df.head()

,X,y
0,data/76878/768781748033335296-1.jpg,2
1,data/80174/801747152955326465-1.jpg,0
2,data/76899/768993807845163008-3.jpg,2
3,data/80162/801624637314629634-1.jpg,0
4,data/78083/780835980597194753-1.jpg,2


In [4]:
print(train_data_df.shape)
print(valid_data_df.shape)
print(test_data_df.shape)

print(train_data_df.shape[0] + valid_data_df.shape[0] + test_data_df.shape[0])
sample_df = test_data_df[:5]
sample_df

(1842, 2)
(255, 2)
(255, 2)
2352


,X,y
0,data/76878/768781748033335296-1.jpg,2
1,data/80174/801747152955326465-1.jpg,0
2,data/76899/768993807845163008-3.jpg,2
3,data/80162/801624637314629634-1.jpg,0
4,data/78083/780835980597194753-1.jpg,2


In [5]:
# load TTD
ttd_basedir="/home/tutysara/src/myprojects/senti/dataset"
ttd_data_idx_path = ttd_basedir+ "/twitter_five_agrees.txt"
col_names = ["X", "y"]
ttd_data_df = pd.read_csv(ttd_data_idx_path, sep=" ", header=None, names=col_names)
ttd_data_df.X = ttd_data_df.X.apply(lambda x: ttd_basedir+"/Agg_AMT_Candidates/"+x)
print(ttd_data_df.shape)
ttd_data_df.head()

(882, 2)


,X,y
0,/home/tutysara/src/myprojects/senti/dataset/Agg_AMT_Candidates/28800526...,0
1,/home/tutysara/src/myprojects/senti/dataset/Agg_AMT_Candidates/33547480...,1
2,/home/tutysara/src/myprojects/senti/dataset/Agg_AMT_Candidates/27176682...,1
3,/home/tutysara/src/myprojects/senti/dataset/Agg_AMT_Candidates/10096570...,1
4,/home/tutysara/src/myprojects/senti/dataset/Agg_AMT_Candidates/13247920...,1


In [6]:
# convert and load images
from keras.preprocessing import image                  
from tqdm import tqdm
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in img_paths]
    return np.vstack(list_of_tensors)

In [7]:
#definition of data generator
def df_data_generator(df, batch_size=32, num_classes=3, shuffle=False, basedir=".", transformer=None, progress=True):    
    while True:
        if shuffle:
            df = df.sample(frac=1)
            
        X_file_name = df.X.apply(lambda x: basedir+"/"+x) 
        y = np_utils.to_categorical(df.y, num_classes)
    # infinitely serve batches
        max_range = (1+ df.shape[0]//batch_size)
        for i in range(max_range):
            s = time.time()
            if transformer:
                yield transformer(X_file_name[i*batch_size : (i+1)*batch_size]).astype('float32'), y[i*batch_size : (i+1)*batch_size]
            else:
                yield X_file_name[i*batch_size : (i+1)*batch_size], y[i*batch_size : (i+1)*batch_size]
            if progress and max_range>1:
                print("df.gen Iteration {i}/{t} took {s:.2f}s".format(i=i, t=max_range, s=(time.time()-s)))                   
def img_to_tensor_transformer(x):
    return paths_to_tensor(x)


In [8]:
import bcolz
def bcolz_writer(gen, steps, dirname, progress=False ):
    X = None; y = None
    data_dir = dirname + "_data.bclz"
    label_dir = dirname + "_labels.bclz"
    for i in range(steps):
        X_batch, y_batch = next(gen)
        if i== 0:
            X = bcolz.carray(X_batch, rootdir=data_dir, mode='w')
            y = bcolz.carray(y_batch, rootdir=label_dir, mode='w') 
        else:
            X.append(X_batch)
            y.append(y_batch)
    X.flush()
    y.flush()
    return X, y

In [9]:
bnf_train_name = 'bottleneck_features_mobilenet_train'
bnf_valid_name = 'bottleneck_features_mobilenet_valid'
bnf_test_name = 'bottleneck_features_mobilenet_test' 

In [10]:
train_name = 'train_data'
valid_name = 'valid_data'
test_name = 'test_data'

In [11]:
## load train, test, and validation datasets
## load and convert to bcolz format for fast reading

import time

batch_size=256
num_classes = 3
train_gen = df_data_generator(train_data_df,
                              batch_size=batch_size, 
                              transformer=img_to_tensor_transformer, 
                              basedir=basedir,
                              num_classes=num_classes)

valid_gen = df_data_generator(valid_data_df,
                              batch_size=batch_size, 
                              transformer=img_to_tensor_transformer, 
                              basedir=basedir,
                              num_classes=num_classes)

test_gen = df_data_generator(test_data_df,
                             batch_size=batch_size, 
                             transformer=img_to_tensor_transformer, 
                             basedir=basedir,
                             num_classes=num_classes)

s= time.time()
!rm -rf 'valid_data*bclz'
valid_data,  valid_labels = bcolz_writer(gen=valid_gen, 
                        steps=(1+(valid_data_df.shape[0]//batch_size)),
                        dirname=valid_name)
print("Took {:.2f} seconds to calculate valid_data".format(time.time()-s))

s= time.time()
!rm -rf 'test_data*bclz'
test_data,  test_labels = bcolz_writer(gen=test_gen, 
                        steps=(1+(test_data_df.shape[0]//batch_size)),
                        dirname=test_name)
print("Took {:.2f} seconds to calculate test_data".format(time.time()-s))

s= time.time()
!rm -rf 'train_data*bclz'
train_data,  train_labels = bcolz_writer(gen=train_gen, 
                        steps=(1+(train_data_df.shape[0]//batch_size)),
                        dirname=train_name)
print("Took {:.2f} seconds to calculate train_data".format(time.time()-s))

Took 7.53 seconds to calculate valid_data
Took 7.07 seconds to calculate test_data
df.gen Iteration 0/8 took 7.38s
df.gen Iteration 1/8 took 5.95s
df.gen Iteration 2/8 took 6.76s
df.gen Iteration 3/8 took 6.89s
df.gen Iteration 4/8 took 6.31s
df.gen Iteration 5/8 took 6.55s
df.gen Iteration 6/8 took 6.47s
Took 47.70 seconds to calculate train_data


In [12]:
print(valid_data.shape)
print(test_data.shape)
print(train_data.shape)

print(valid_labels.shape)
print(test_labels.shape)
print(train_labels.shape)

(255, 224, 224, 3)
(255, 224, 224, 3)
(1842, 224, 224, 3)
(255, 3)
(255, 3)
(1842, 3)


In [13]:
# read from disk and check size
valid_data = bcolz.carray(rootdir= valid_name+'_data.bclz', mode='r')
test_data = bcolz.carray(rootdir= test_name + '_data.bclz', mode='r')
train_data = bcolz.carray(rootdir= train_name+ '_data.bclz', mode='r')


valid_labels = bcolz.carray(rootdir= valid_name+'_labels.bclz', mode='r')
test_labels = bcolz.carray(rootdir= test_name + '_labels.bclz', mode='r')
train_labels = bcolz.carray(rootdir= train_name+ '_labels.bclz', mode='r')

print(valid_data.shape)
print(test_data.shape)
print(train_data.shape)

print(valid_labels.shape)
print(test_labels.shape)
print(train_labels.shape)

(255, 224, 224, 3)
(255, 224, 224, 3)
(1842, 224, 224, 3)
(255, 3)
(255, 3)
(1842, 3)


In [14]:
def bcolz_prediction_writer(gen, steps, model, preprocess, dirname ):
    X = None; y = None
    data_dir = dirname + "_data.bclz"
    label_dir = dirname + "_labels.bclz"
    for i in range(steps):
        X_batch, y_batch = next(gen)
        if preprocess:
            X_out = model.predict(preprocess(X_batch))
        else:
            X_out = model.predict(X_batch)
        if i== 0:
            X = bcolz.carray(X_out, rootdir=data_dir, mode='w')
            y = bcolz.carray(y_batch, rootdir=label_dir, mode='w') 
        else:
            X.append(X_out)
            y.append(y_batch)
    X.flush()
    y.flush()
    return X, y

In [15]:
def bcolz_data_generator(bclz_data, bclz_labels, batch_size=32, progress=False):
    while True:
        max_range = (1 + bclz_data.shape[0]//batch_size)
        for i in range(max_range):
            s = time.time()
            yield bclz_data[i*batch_size : (i+1)*batch_size], bclz_labels[i*batch_size : (i+1)*batch_size]
            if progress and max_range>1:
                print("bcolz.gen Iteration {i}/{t} took {s:.2f}s".format(i=i, t=max_range, s=(time.time()-s))) 

In [16]:
## make a generator of loaded data
batch_size = 256
valid_data_gen = bcolz_data_generator(valid_data, valid_labels, batch_size=batch_size, progress=True)
test_data_gen = bcolz_data_generator(test_data, test_labels, batch_size=batch_size, progress=True)
train_data_gen = bcolz_data_generator(train_data, train_labels, batch_size=batch_size, progress=True)

In [17]:
## Transform data and save it in bottleneck feacture format

from keras.applications.mobilenet import MobileNet
from keras.applications.mobilenet import preprocess_input as mobilenet_preprocess_input
import time

mobilenet_feature_ext = MobileNet(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
mobilenet_feature_ext._make_predict_function()

s= time.time()
!rm -rf 'bottleneck_features_mobilenet_valid*bclz'
bnf_valid_data,  bnf_valid_labels = bcolz_prediction_writer(gen=valid_data_gen, 
                        steps=(1+(valid_data_df.shape[0]//batch_size)),
                        model=mobilenet_feature_ext,
                        preprocess= mobilenet_preprocess_input,
                        dirname=bnf_valid_name)
print("Took {:.2f} seconds to calculate bnf_valid_data".format(time.time()-s))

s= time.time()
!rm -rf 'bottleneck_features_mobilenet_test*bclz'
bnf_test_data,  bnf_test_labels = bcolz_prediction_writer(gen=test_data_gen, 
                        steps=(1+(test_data_df.shape[0]//batch_size)),
                        model=mobilenet_feature_ext,
                        preprocess= mobilenet_preprocess_input,
                        dirname=bnf_test_name)
print("Took {:.2f} seconds to calculate bnf_test_data".format(time.time()-s))

s= time.time()
!rm -rf 'bottleneck_features_mobilenet_train*bclz'
bnf_train_data,  bnf_train_labels = bcolz_prediction_writer(gen=train_data_gen, 
                        steps=(1+(train_data_df.shape[0]//batch_size)),
                        model=mobilenet_feature_ext,
                        preprocess= mobilenet_preprocess_input,
                        dirname=bnf_train_name)
print("Took {:.2f} seconds to calculate bottleneck_features_mobilenet_train".format(time.time()-s))


Took 2.20 seconds to calculate bnf_valid_data
Took 1.25 seconds to calculate bnf_test_data
bcolz.gen Iteration 0/8 took 1.13s
bcolz.gen Iteration 1/8 took 1.06s
bcolz.gen Iteration 2/8 took 1.05s
bcolz.gen Iteration 3/8 took 1.05s
bcolz.gen Iteration 4/8 took 1.05s
bcolz.gen Iteration 5/8 took 1.05s
bcolz.gen Iteration 6/8 took 1.05s
Took 7.79 seconds to calculate bottleneck_features_mobilenet_train


In [18]:
print(bnf_valid_data.shape)
print(bnf_test_data.shape)
print(bnf_train_data.shape)

print(bnf_valid_labels.shape)
print(bnf_test_labels.shape)
print(bnf_train_labels.shape)

(255, 7, 7, 1024)
(255, 7, 7, 1024)
(1842, 7, 7, 1024)
(255, 3)
(255, 3)
(1842, 3)


In [19]:
## Read it back from disk and check size
bnf_valid_data = bcolz.carray(rootdir='bottleneck_features_mobilenet_valid_data.bclz', mode='r')
bnf_test_data = bcolz.carray(rootdir='bottleneck_features_mobilenet_test_data.bclz', mode='r')
bnf_train_data = bcolz.carray(rootdir='bottleneck_features_mobilenet_train_data.bclz', mode='r')

bnf_valid_labels = bcolz.carray(rootdir='bottleneck_features_mobilenet_valid_labels.bclz', mode='r')
bnf_test_labels = bcolz.carray(rootdir='bottleneck_features_mobilenet_test_labels.bclz', mode='r')
bnf_train_labels = bcolz.carray(rootdir='bottleneck_features_mobilenet_train_labels.bclz', mode='r')

print(bnf_valid_data.shape)
print(bnf_test_data.shape)
print(bnf_train_data.shape)


print(bnf_valid_labels.shape)
print(bnf_test_labels.shape)
print(bnf_train_labels.shape)

(255, 7, 7, 1024)
(255, 7, 7, 1024)
(1842, 7, 7, 1024)
(255, 3)
(255, 3)
(1842, 3)


In [20]:
## make a generator of loaded data
batch_size = 256
bnf_train_gen =bcolz_data_generator(bnf_train_data, bnf_train_labels, batch_size=batch_size)
bnf_valid_gen =bcolz_data_generator(bnf_valid_data, bnf_valid_labels, batch_size=batch_size)
bnf_test_gen =bcolz_data_generator(bnf_test_data, bnf_test_labels, batch_size=batch_size) 

In [21]:
import keras.backend as K
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, Reshape
from keras import regularizers

alpha = 1
dropout=1e-3

if K.image_data_format() == 'channels_first':
    shape = (int(1024 * alpha), 1, 1)
else:
    shape = (1, 1, int(1024 * alpha))

classes = num_classes

top_model = Sequential()
top_model.add(GlobalAveragePooling2D(input_shape=(7, 7, 1024)))
top_model.add(Reshape(shape, name='reshape_1'))
top_model.add(Dropout(dropout, name='dropout'))
top_model.add(Conv2D(classes, (1, 1),
           padding='same', name='conv_preds'))
top_model.add(Activation('softmax', name='act_softmax'))
top_model.add(Reshape((classes,), name='reshape_2'))

top_model.compile(optimizer='adam',
              loss='categorical_crossentropy', metrics=['accuracy'])
top_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_1 ( (None, 1024)              0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 1, 1, 1024)        0         
_________________________________________________________________
dropout (Dropout)            (None, 1, 1, 1024)        0         
_________________________________________________________________
conv_preds (Conv2D)          (None, 1, 1, 3)           3075      
_________________________________________________________________
act_softmax (Activation)     (None, 1, 1, 3)           0         
_________________________________________________________________
reshape_2 (Reshape)          (None, 3)                 0         
Total params: 3,075
Trainable params: 3,075
Non-trainable params: 0
_________________________________________________________________


In [22]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.topmodel.mobilenet.hdf5', verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=2, verbose=1)
top_model.fit_generator(bnf_train_gen,
          steps_per_epoch= (1 + (bnf_train_data.shape[0]// batch_size)),
          epochs=15,
          validation_data=bnf_valid_gen,
          validation_steps= (1 + (bnf_valid_data.shape[0] // batch_size)),
          callbacks=[early_stopping, checkpointer])

Epoch 1/15
8/8 [==============================] - 1s 124ms/step - loss: 1.4993 - acc: 0.3099 - val_loss: 1.3774 - val_acc: 0.3255
Epoch 2/15
8/8 [==============================] - 1s 74ms/step - loss: 1.3222 - acc: 0.3392 - val_loss: 1.3250 - val_acc: 0.3529
Epoch 3/15
8/8 [==============================] - 1s 87ms/step - loss: 1.2422 - acc: 0.3922 - val_loss: 1.3078 - val_acc: 0.3529
Epoch 4/15
8/8 [==============================] - 1s 86ms/step - loss: 1.1787 - acc: 0.3971 - val_loss: 1.2567 - val_acc: 0.3882
Epoch 5/15
8/8 [==============================] - 1s 88ms/step - loss: 1.1343 - acc: 0.4234 - val_loss: 1.2459 - val_acc: 0.3961
Epoch 6/15
8/8 [==============================] - 1s 86ms/step - loss: 1.0978 - acc: 0.4449 - val_loss: 1.2234 - val_acc: 0.4039
Epoch 7/15
8/8 [==============================] - 1s 87ms/step - loss: 1.0622 - acc: 0.4584 - val_loss: 1.2117 - val_acc: 0.4078
Epoch 8/15
8/8 [==============================] - 1s 88ms/step - loss: 1.0343 - acc: 0.4872 - va

In [31]:
def prediction_from_gen(gen, steps, model, dirname ):
    y_true = None; y_pred = None
    yt_dir = dirname + "_y_true.bclz"
    yp_dir = dirname + "_y_pred.bclz"
    for i in range(steps):
        X_batch, y_batch = next(gen)
        y_out = model.predict(X_batch)
        if i== 0:
            y_true = bcolz.carray(y_batch, rootdir=yt_dir, mode='w')
            y_pred = bcolz.carray(y_out, rootdir=yp_dir, mode='w') 
        else:
            y_true.append(y_batch)
            y_pred.append(y_out)
    y_true.flush()
    y_pred.flush()
    return y_true, y_pred

y_true, y_pred = prediction_from_gen(gen=bnf_test_gen,
                                     steps=(1 + (test_data_df.shape[0] // batch_size)),
                                     model=top_model,
                                     dirname="bottleneck_features_mobilenet_test"
                                    )


In [24]:
print(y_true.shape)
print(y_pred.shape)

(255, 3)
(255, 3)


In [32]:
# report test accuracy
test_accuracy = 100*np.sum(np.argmax(y_pred, axis=1)==np.argmax(y_true, axis=1))/len(y_true)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 37.6471%


In [33]:
# report test accuracy on TTD
ttd_X1 = ttd_data_df.X.as_matrix()
ttd_X2 = mobilenet_preprocess_input(paths_to_tensor(ttd_X1))
ttd_X = mobilenet_feature_ext.predict(ttd_X2)

In [34]:
ttd_y_pred = top_model.predict(ttd_X)
ttd_y_pred_two_classes=ttd_y_pred[:,[0,2]]
ttd_y_true = ttd_data_df.y.as_matrix()

In [35]:
print(ttd_X.shape)
print(ttd_y_pred.shape)
print(ttd_y_true.shape) 

(882, 7, 7, 1024)
(882, 3)
(882,)


In [36]:
ttd_test_accuracy = 100*np.sum(np.argmax(ttd_y_pred_two_classes, axis=1)==ttd_y_true)/len(ttd_y_true)
print('TTD Test accuracy: %.4f%%' % ttd_test_accuracy)

TTD Test accuracy: 59.1837%


In [37]:
# fine tune on full model
from keras import optimizers
from keras.callbacks import EarlyStopping, ModelCheckpoint

mobilenet_model = MobileNet(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

# CREATE AN "REAL" MODEL FROM Mobilenet
# BY COPYING ALL THE LAYERS OF Mobilenet
model = Sequential()
for l in mobilenet_model.layers:
    model.add(l)


# CONCATENATE THE TWO MODELS
model.add(top_model)

# LOCK THE TOP CONV LAYERS
for layer in model.layers:
    layer.trainable = False

# COMPILE THE MODEL
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (Activation)      (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________________________
conv_dw_1_relu (Activation)  (None, 112, 112, 32)      0         
__________

In [39]:
# load train, test, and validation datasets
batch_size = 256
valid_data_gen = bcolz_data_generator(valid_data, valid_labels, batch_size=batch_size, progress=False)
test_data_gen = bcolz_data_generator(test_data, test_labels, batch_size=batch_size, progress=False)
train_data_gen = bcolz_data_generator(train_data, train_labels, batch_size=batch_size, progress=False)

from keras.callbacks import EarlyStopping, ModelCheckpoint
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.fullmodel.mobilenet.hdf5', verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=2, verbose=1)
model.fit_generator(train_data_gen,
          steps_per_epoch= (1+ train_data_df.shape[0]// batch_size),
          epochs=2,
          validation_data=valid_data_gen,
          validation_steps= (1+ valid_data_df.shape[0] // batch_size),
          callbacks=[early_stopping, checkpointer])

Epoch 1/2
8/8 [==============================] - 6s 713ms/step - loss: 0.9761 - acc: 0.5290 - val_loss: 1.2150 - val_acc: 0.4549
Epoch 2/2
8/8 [==============================] - 5s 664ms/step - loss: 0.9778 - acc: 0.5252 - val_loss: 1.2181 - val_acc: 0.4549


In [40]:

y_true, y_pred = prediction_from_gen(gen=test_data_gen,
                                     steps=(1 + (test_data_df.shape[0] // batch_size)),
                                     model=model,
                                     dirname="mobilenet_test"
                                    )

In [41]:
# report test accuracy
test_accuracy = 100*np.sum(np.argmax(y_pred, axis=1)==np.argmax(y_true, axis=1))/len(y_true)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 45.0980%


In [42]:
# report test accuracy on TTD
ttd_X1 = ttd_data_df.X.as_matrix()
ttd_X2 = mobilenet_preprocess_input(paths_to_tensor(ttd_X1))
ttd_y_pred = model.predict(ttd_X2)

ttd_y_pred_two_classes=ttd_y_pred[:,[0,2]]
ttd_y_true = ttd_data_df.y.as_matrix()

In [43]:
ttd_test_accuracy = 100*np.sum(np.argmax(ttd_y_pred_two_classes, axis=1)==ttd_y_true)/len(ttd_y_true)
print('TTD Test accuracy: %.4f%%' % ttd_test_accuracy)

TTD Test accuracy: 34.1270%
